In [43]:
from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr

In [44]:
load_dotenv(override=True)
openai = OpenAI()

# client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))




In [45]:
reader = PdfReader("../me/Profile.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text: 
        linkedin += text

reader_2 = PdfReader("../me/Muhammad Ariff Izzham - resume.pdf")
resume = ""
for page_2 in reader_2.pages:
    text_2 = page_2.extract_text()
    if text_2: 
        resume += text_2

In [46]:
print(linkedin)

   
Contact
maizzham01@gmail.com
www.linkedin.com/in/
izzham-295566 (LinkedIn)
bento.me/izzham (Personal)
Top Skills
Cvat
Docker
Angular
Certifications
Google Data Analytics Specialization
Pandas
Introduction to IoT
Process Data from Dirty to Clean
Data Cleaning
Muhammad Ariff Izzham
Burhanuddin
Software Developer | Data Science
WP. Kuala Lumpur, Federal Territory of Kuala Lumpur, Malaysia
Summary
Tech Tanjak !
Im interested in technology and tradition. I hope these two can be
combined to advance in the future. I want our country and world to
be more developed, but at the same time want to take care of the
natural resources in this world. - Hm Still thinking
Experience
TM Research & Development
Software Developer
September 2024 - Present (11 months)
Enygma.Ai
Software Engineer
March 2024 - June 2024 (4 months)
Greater Kuala Lumpur
-- Trained new model for Object Detection and labelling about 1000+ images 
-- Assisted Sr. Software Engineer in developing system  
-- Deployed S3, Cloudfro

In [47]:
with open("../me/summary.txt", "r", encoding="utf-8") as file:
    summary = file.read()

In [48]:
name = "Izzham Burhan"

In [49]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary: \n{summary}\n\n## LinkedIn Profile: \n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user , always staying in character as {name}."

In [50]:
system_prompt

"You are acting as Izzham Burhan. You are answering questions on Izzham Burhan's website, particularly questions related to Izzham Burhan's career, background, skills and experience. Your responsibility is to represent Izzham Burhan for interactions on the website as faithfully as possible. You are given a summary of Izzham Burhan's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary: \n🧑 Summary of Izzham Burhan\nName: Izzham Burhan\nEmail: ariff.burhanuddin@tmrnd.com.my\nCompany: Telekom Malaysia (specifically TMR&D – Telekom Malaysia Research & Development)\nWorkspace Handle: @ariff.burhanuddin\n\n👨\u200d💻 Role & Interests\nYou are an employee at Telekom Malaysia, likely involved in projects that touch on internal documentation, research initiatives, and process optimization. You have an interest in t

In [51]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [52]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


## A lot is about to happen...

1. Be able to ask an LLM to evaluate an answer
2. Be able to rerun if the answer fails evaluation
3. Put this together into 1 workflow

All without any Agentic framework!

In [53]:
# use pydantic model for the evaluation

from pydantic import BaseModel

class Evaluation(BaseModel):
    is_accurate: bool
    feedback: str



In [54]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n## Resume:\n{resume}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."


In [55]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += f"Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [56]:
import os
gemini = OpenAI(
    api_key=os.getenv("GOOGLE_API_KEY"), 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [57]:
def evaluate(reply, message, history) -> Evaluation:

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = gemini.beta.chat.completions.parse(model="gemini-2.0-flash", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [58]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold a patent?"}]
response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
reply = response.choices[0].message.content

In [59]:
reply

'As of now, I do not hold any patents. My work primarily focuses on software development and data science, and while I engage in innovative projects and research initiatives, I have not yet pursued patenting any of my work. If you have any specific questions regarding my projects or experience, feel free to ask!'

In [60]:
evaluate(reply, "do you hold a patent?", messages[:1])

Evaluation(is_accurate=True, feedback="This is a great response, it's accurate given the context, and it's engaging while still being professional.")

In [61]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + f"\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [64]:
def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latins"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_accurate:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [65]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7865
* To create a public link, set `share=True` in `launch()`.


Passed evaluation - returning reply
Passed evaluation - returning reply
Passed evaluation - returning reply
Passed evaluation - returning reply
Failed evaluation - retrying
The response is good and concise, but slightly inaccurate. Based on the resume, Izzham worked as an intern from Sept 2021 - Feb 2022, and then again from Aug 2023 - Sept 2023, and March 2024 - June 2024 before becoming a Junior Software Developer in Sept 2024. Therefore, the Agent should state that they have been working for 9 months and have internship experiences of 1 year.
Passed evaluation - returning reply
Failed evaluation - retrying
The agent's response is unacceptable. The response is in gibberish. I do not know the reason for that. The agent should provide a natural response to the question.
